# The Data that I will use for my Investment Strategy Project

In [1]:
#Importing Libraries

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

In [2]:
# Credentials
CLIENT_ID = 'FVDQ13V4GEIKKHIASW1FL1LXL1NRQAMH100XNXD0KGHMAMCI' # your Foursquare ID
CLIENT_SECRET = '11JWZ3YYGWSAMTPNCRUKK2DAND5QFBWMHC2R4ZXRQX0IW4GX' # your Foursquare Secret
VERSION = '20180604'

In [3]:
latitude=37.983810
longitude=23.727539
radius = 2000
LIMIT = 50

In [4]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=FVDQ13V4GEIKKHIASW1FL1LXL1NRQAMH100XNXD0KGHMAMCI&client_secret=11JWZ3YYGWSAMTPNCRUKK2DAND5QFBWMHC2R4ZXRQX0IW4GX&ll=37.98381,23.727539&v=20180604&radius=2000&limit=50'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf134cd6a6071493ff0e8c4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Metaxourgeio',
  'headerFullLocation': 'Metaxourgeio, Athens',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 233,
  'suggestedBounds': {'ne': {'lat': 38.001810018000015,
    'lng': 23.750333670031498},
   'sw': {'lat': 37.96580998199998, 'lng': 23.704744329968502}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e822369f5b91d2e46d16f27',
       'name': 'National Theatre of Greece (Εθνικό Θέατρο)',
       'location': {'address': 'Αγίου Κωνσταντίνου 22-24',
        'lat': 37.98474217257332,
        'lng': 23.725541110739435,
        'labeledLatLngs': [

In [6]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e822369f5b91d2e46d16f27',
  'name': 'National Theatre of Greece (Εθνικό Θέατρο)',
  'location': {'address': 'Αγίου Κωνσταντίνου 22-24',
   'lat': 37.98474217257332,
   'lng': 23.725541110739435,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.98474217257332,
     'lng': 23.725541110739435}],
   'distance': 203,
   'postalCode': '104 37',
   'cc': 'GR',
   'city': 'Αθήνα',
   'state': 'Αττική',
   'country': 'Ελλάδα',
   'formattedAddress': ['Αγίου Κωνσταντίνου 22-24',
    '104 37 Αθήνα, Αττική',
    'Ελλάδα']},
  'categories': [{'id': '4bf58dd8d48988d137941735',
    'name': 'Theater',
    'pluralName': 'Theaters',
    'shortName': 'Theater',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 

In [10]:
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,National Theatre of Greece (Εθνικό Θέατρο),Theater,Αγίου Κωνσταντίνου 22-24,GR,Αθήνα,Ελλάδα,NaN,203,"[Αγίου Κωνσταντίνου 22-24, 104 37 Αθήνα, Αττικ...","[{'label': 'display', 'lat': 37.98474217257332...",37.984742,23.725541,NaN,104 37,Αττική,4e822369f5b91d2e46d16f27
1,Στάνη Γαλακτοπωλείο,Dessert Shop,Μαρίκας Κοτοπούλη 10,GR,Αθήνα,Ελλάδα,Βερανζέρου,186,"[Μαρίκας Κοτοπούλη 10 (Βερανζέρου), 104 32 Αθή...","[{'label': 'display', 'lat': 37.98546282764882...",37.985463,23.727908,Metaxourgeio,104 32,Αττική,4c038ca13f03b713b6fb5141
2,Loukoumades Ktistakis (Λουκουμάδες Χανίων Θ. Κ...,Donut Shop,Σωκράτους 59,GR,Αθήνα,Ελλάδα,NaN,69,"[Σωκράτους 59, 104 31 Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98393592722866...",37.983936,23.726768,NaN,104 31,Αττική,4d4bdade1ae43704f77ced60
3,Taf Coffee,Coffee Shop,Εμμανουήλ Μπενάκη 7,GR,Αθήνα,Ελλάδα,NaN,293,"[Εμμανουήλ Μπενάκη 7, 106 78 Αθήνα, Αττική, Ελ...","[{'label': 'display', 'lat': 37.98345088308742...",37.983451,23.730853,NaN,106 78,Αττική,4b64103df964a520759d2ae3
4,Τράτα,Fish Taverna,Νικηταρά 9,GR,Αθήνα,Ελλάδα,Θεμιστοκλέους 8,301,"[Νικηταρά 9 (Θεμιστοκλέους 8), 106 78 Αθήνα, Α...","[{'label': 'display', 'lat': 37.98421877664627...",37.984219,23.730939,"Omonia, Athens, Attica",106 78,Αττική,4df4a6d218a88611c6c0c27e
5,Fabrica De Vino,Wine Bar,Εμμανουήλ Μπενάκη 3,GR,Αθήνα,Ελλάδα,NaN,234,"[Εμμανουήλ Μπενάκη 3, 105 64 Αθήνα, Αττική, Ελ...","[{'label': 'display', 'lat': 37.98299091458443...",37.982991,23.729996,NaN,105 64,Αττική,4df7793ee4cd50071443c944
6,Της Θεάτρου Το Στέκι,Kafenio,7 Theatrou St,GR,Αθήνα,Ελλάδα,NaN,393,"[7 Theatrou St, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98070281296481...",37.980703,23.725394,NaN,NaN,Αττική,55156e65498eb8bdba7fd799
7,Mokka,Coffee Shop,Αθηνάς 44,GR,Αθήνα,Ελλάδα,Σοφοκλέους,330,"[Αθηνάς 44 (Σοφοκλέους), 105 53 Αθήνα, Αττική,...","[{'label': 'display', 'lat': 37.98085177886126...",37.980852,23.727177,NaN,105 53,Αττική,4eedcdb56c25c7f9d26ccc4f
8,Mr Bean Coffee Brewers,Coffee Shop,Εμ. Μπενακη 20,GR,Αθήνα,Ελλάδα,NaN,326,"[Εμ. Μπενακη 20, Αθήνα, Αττική, Ελλάδα]","[{'label': 'display', 'lat': 37.98360955042584...",37.983610,23.731250,NaN,NaN,Αττική,56ed54a1498e1bc48d70301d
9,Καραμανλίδικα του Φάνη,Meze Restaurant,Σωκράτους 1,GR,Αθήνα,Ελλάδα,Ευριπίδου 52,434,"[Σωκράτους 1 (Ευριπίδου 52), 105 54 Αθήνα, Αττ...","[{'label': 'display', 'lat': 37.98016872811729...",37.980169,23.725757,Ψυρρή,105 54,Αττική,5374e6b9498e90c75ce9c0b4


In [11]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\thwma\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.0.1               |           py37_0         722 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.3.9           |           py37_0         149 KB  conda-forge
    conda-4.6.14               |           py37_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  altair             conda-forge/win-64::altair-3.0.1



==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda


'ET' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map 
# add a red circle marker to represent the Center
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)

In [17]:
# add the venues as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=4,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)


In [18]:
# display map
venues_map2